<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/af/examples/af_pseudo_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AF_pseudo_diffusion + proteinMPNN
Hacking AlphaFold to be a diffusion model (for backbone generation). At each step add logits from proteinMPNN.


**WARNING**: This notebook is experimental, designed a control. Not intended for practical use at this stage.

In [ ]:
#@title setup
%%time
import os
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.7/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar")
  os.system("tar -xf alphafold_params_2022-03-02.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from colabdesign.mpnn import mk_mpnn_model
from IPython.display import HTML
from google.colab import files
import numpy as np

In [ ]:
#@title initialize the model
length = 100 #@param {type:"integer"}
clear_mem()
af_model = mk_afdesign_model(protocol="hallucination", use_templates=True)
af_model.prep_inputs(length=length)
mpnn_model = mk_mpnn_model()

In [ ]:
#@title run protocol
iterations = 100 #@param ["50", "100"] {type:"raw"}
noise_xyz = 1.0 #@param ["0.0", "1.0", "2.0"] {type:"raw"}
rm_template_seq = True #@param {type:"boolean"}
#@markdown proteinMPNN options
mpnn_fract = 0.1 #@param ["0", "0.1"] {type:"raw"}
mpnn_mode = "conditional" #@param ["conditional", "unconditional"]
#@markdown - set `mpnn_fract` to `0` to disable proteinmpnn

af_model.restart(mode="gumbel")
af_model._inputs["rm_template_seq"] = rm_template_seq
L = sum(af_model._lengths)
af_model._inputs["batch"] = {"aatype":np.zeros(L).astype(int),
                             "all_atom_mask":np.ones((L,37)),
                             "all_atom_positions":np.random.normal(size=(L,37,3))}

for k in range(iterations):
  # add noise
  if noise_xyz > 0:
    n = np.random.normal(size=(L,37,3)) * noise_xyz * (1-k/iterations)
    af_model._inputs["batch"]["all_atom_positions"] += n

  # denoise
  aux = af_model.predict(return_aux=True, verbose=False)

  # update inputs
  af_model._inputs["batch"]["aatype"] = af_model.aux["seq"]["hard"].argmax(-1)[0]
  af_model._inputs["batch"]["all_atom_positions"] = af_model.aux["atom_positions"].copy()

  # add logits from proteinmpnn at each stage
  if mpnn_fract > 0:
    mpnn_model.get_af_inputs(af_model)
    opt = {} if mpnn_mode == "conditional" else {"ar_mask":np.zeros((L,L))}
    mpnn_out = mpnn_model.score(**opt)
    aux["log"]["mpnn"] = mpnn_out["score"]
    mpnn_logits = mpnn_out["logits"][:,:20]
    m = mpnn_fract * k/iterations
    af_model._params["seq"] = (1-m) * af_model._params["seq"] + m * mpnn_logits
  
  # save results
  af_model._save_results(aux)
  af_model._k += 1

In [ ]:
af_model.plot_pdb()

In [ ]:
af_model.save_pdb(f"tmp.pdb")
af_model.get_seqs()

In [ ]:
HTML(af_model.animate(dpi=100))